<span style="color:rgb(60, 64, 67);font-family:Roboto, Arial, sans-serif;letter-spacing:0.2px;">მონაცემთა ბაზის კონსტრუირება (ინდექსების დასმა და ცხრილების შორის კავშირების უზრუნველყოფა). (4 ქულა)</span>

In [5]:
CREATE DATABASE ImaginaryWorld;
USE ImaginaryWorld;

: Msg 911, Level 16, State 1, Line 2
Database 'ImaginaryWorld' does not exist. Make sure that the name is entered correctly.

Total execution time: 00:00:00.015

In [28]:
CREATE TABLE Creatures (
    CreatureID INT PRIMARY KEY IDENTITY(1,1),
    Name NVARCHAR(100) NOT NULL,
    Type NVARCHAR(50) NOT NULL,
    Habitat NVARCHAR(100),
    Age INT,
    Description NVARCHAR(MAX)
);

Commands completed successfully.

Total execution time: 00:00:00.073

In [29]:
CREATE TABLE Spells (
    SpellID INT PRIMARY KEY IDENTITY(1,1),
    Name NVARCHAR(100) NOT NULL,
    Description NVARCHAR(MAX),
    PowerLevel INT
);

Commands completed successfully.

Total execution time: 00:00:00.016

In [30]:
CREATE TABLE Locations (
    LocationID INT PRIMARY KEY IDENTITY(1,1),
    Name NVARCHAR(100) NOT NULL,
    TerrainType NVARCHAR(50),
    Climate NVARCHAR(50),
    Description NVARCHAR(MAX)
);

Commands completed successfully.

Total execution time: 00:00:00.022

In [31]:
CREATE TABLE CreaturesSpells (
    CreatureID INT,
    SpellID INT,
    PRIMARY KEY (CreatureID, SpellID),
    FOREIGN KEY (CreatureID) REFERENCES Creatures(CreatureID),
    FOREIGN KEY (SpellID) REFERENCES Spells(SpellID)
);

Commands completed successfully.

Total execution time: 00:00:00.018

In [32]:
CREATE TABLE CreatureLocations (
    CreatureID INT,
    LocationID INT,
    PRIMARY KEY (CreatureID, LocationID),
    FOREIGN KEY (CreatureID) REFERENCES Creatures(CreatureID),
    FOREIGN KEY (LocationID) REFERENCES Locations(LocationID)
);

Commands completed successfully.

Total execution time: 00:00:00.016

In [ ]:
-- 1. Modifying a table (Adding a new column to Creatures table)
ALTER TABLE Creatures
ADD SpecialAbility NVARCHAR(100);

-- 2. Altering a table (Changing the data type of a column in Creatures table)
ALTER TABLE Creatures
ALTER COLUMN Description NVARCHAR(500);

-- 3. Deleting a table (Dropping the CreatureLocations table)
DROP TABLE CreatureLocations;

-- 4. Renaming a table (Renaming the Creatures table to MythicalCreatures)
EXEC sp_rename 'Creatures', 'MythicalCreatures';

-- 5. Dropping a column from a table (Removing the Age column from MythicalCreatures table)
ALTER TABLE MythicalCreatures
DROP COLUMN Age;

In [33]:
CREATE INDEX IX_Creature_Name ON Creatures(Name);
CREATE INDEX IX_Spell_Name ON Spells(Name);
CREATE INDEX IX_Location_Name ON Locations(Name);
CREATE INDEX IX_CreatureLocations_CreatureID_LocationID ON CreatureLocations(CreatureID, LocationID);
CREATE INDEX IX_Spell_PowerLevel ON Spells(PowerLevel);
-- CREATE CLUSTERED INDEX CIX_Location_LocationID ON Locations(LocationID);
CREATE NONCLUSTERED INDEX NCIX_Location_Terrain_Climate ON Locations(TerrainType, Climate);

Commands completed successfully.

Total execution time: 00:00:00.029

In [ ]:
SELECT Name, Type, Habitat
FROM Creatures
WHERE Type = 'Dragon'
ORDER BY Name;

In [ ]:
SELECT c.Name, c.Type, COUNT(cs.SpellID) AS SpellCount
FROM Creatures c
LEFT JOIN CreaturesSpells cs ON c.CreatureID = cs.CreatureID
GROUP BY c.Name, c.Type;

In [ ]:
SELECT Type, COUNT(*) AS CreatureCount
FROM Creatures
GROUP BY Type
HAVING COUNT(*) > 3;

In [ ]:
SELECT TOP 5 Name, Description
FROM Creatures
ORDER BY CreatureID DESC;

In [ ]:
SELECT Type, MIN(Age) AS MinAge, MAX(Age) AS MaxAge, AVG(Age) AS AvgAge, SUM(Age) AS TotalAge
FROM Creatures
GROUP BY Type;

In [ ]:
SELECT Name, Type
FROM Creatures
WHERE Type = 'Dragon' AND Habitat = 'Mountains';

In [ ]:
CREATE TABLE Events (
    EventID INT PRIMARY KEY IDENTITY(1,1),
    EventName NVARCHAR(100) NOT NULL,
    EventDate DATE NOT NULL
);

INSERT INTO Events (EventName, EventDate)
VALUES ('Dragon Festival', '2023-07-15'),
       ('Wizard Convention', '2023-09-20');

SELECT EventName, EventDate
FROM Events
WHERE EventDate > '2023-08-01';

In [ ]:
SELECT c.Name, s.Name AS SpellName
FROM Creatures c
INNER JOIN CreaturesSpells cs ON c.CreatureID = cs.CreatureID
INNER JOIN Spells s ON cs.SpellID = s.SpellID;


In [ ]:
SELECT c.Name, cs.SpellID
FROM Creatures c
LEFT JOIN CreaturesSpells cs ON c.CreatureID = cs.CreatureID;


In [ ]:
SELECT c.Name, cs.SpellID
FROM CreaturesSpells cs
RIGHT JOIN Creatures c ON cs.CreatureID = c.CreatureID;


In [ ]:
SELECT Name, Type
FROM Creatures
WHERE Name LIKE '%dragon%';


<span style="color:rgb(60, 64, 67);font-family:Roboto, Arial, sans-serif;letter-spacing:0.2px;">ცხრილ(ებ)ში მონაცემების ჩამატება/რედაქტირება პროცედურების (Procedure) გამოყენებით. თუ პროცედურის ბიზნეს ლოგიკა (დანიშნულება) ისეთია, რომ ერთდროულად რამდენიმე ცხრილში ხდება ინფორმაციის შენახვა/რედაქტირება სავალდებულოა ტრანზაქციების გამოყენება.</span>

In [34]:
CREATE PROCEDURE InsertIntoCreatures
    @Name NVARCHAR(100),
    @Type NVARCHAR(50),
    @Habitat NVARCHAR(100),
    @Age INT,
    @Description NVARCHAR(MAX)
AS
BEGIN
    INSERT INTO Creatures (Name, Type, Habitat, Age, Description)
    VALUES (@Name, @Type, @Habitat, @Age, @Description)
END;

Commands completed successfully.

Total execution time: 00:00:00.020

In [35]:
CREATE PROCEDURE InsertIntoSpells
    @Name NVARCHAR(100),
    @Description NVARCHAR(MAX),
    @PowerLevel INT
AS
BEGIN
    INSERT INTO Spells (Name, Description, PowerLevel)
    VALUES (@Name, @Description, @PowerLevel)
END;


Commands completed successfully.

Total execution time: 00:00:00.018

In [36]:
CREATE PROCEDURE InsertIntoLocations
    @Name NVARCHAR(100),
    @TerrainType NVARCHAR(50),
    @Climate NVARCHAR(50),
    @Description NVARCHAR(MAX)
AS
BEGIN
    INSERT INTO Locations (Name, TerrainType, Climate, Description)
    VALUES (@Name, @TerrainType, @Climate, @Description)
END;

Commands completed successfully.

Total execution time: 00:00:00.042

In [37]:
CREATE PROCEDURE InsertIntoCreaturesSpells
    @CreatureID INT,
    @SpellID INT
AS
BEGIN
    INSERT INTO CreaturesSpells (CreatureID, SpellID)
    VALUES (@CreatureID, @SpellID)
END;

Commands completed successfully.

Total execution time: 00:00:00.024

In [38]:
CREATE PROCEDURE InsertIntoCreatureLocations
    @CreatureID INT,
    @LocationID INT
AS
BEGIN
    INSERT INTO CreatureLocations (CreatureID, LocationID)
    VALUES (@CreatureID, @LocationID)
END;

Commands completed successfully.

Total execution time: 00:00:00.006

In [39]:
BEGIN TRANSACTION
    EXEC InsertIntoCreatures 'Dragon', 'Mythical', 'Mountains', 500, 'A mighty fire-breathing creature with scales.';
    EXEC InsertIntoCreatures 'Phoenix', 'Mythical', 'Volcanoes', 1000, 'A mystical bird that rises from its ashes.';

    EXEC InsertIntoSpells 'Fireball', 'Creates a ball of fire', 5;
    EXEC InsertIntoSpells 'Teleportation', 'Moves instantly from one place to another', 7;

    EXEC InsertIntoLocations 'Dragon Lair', 'Mountain', 'Harsh', 'A cave hidden deep within the mountains.';
    EXEC InsertIntoLocations 'Phoenix Roost', 'Volcanic', 'Hot', 'A high cliff surrounded by molten lava.';
    EXEC InsertIntoCreaturesSpells 1, 1; 
    EXEC InsertIntoCreaturesSpells 2, 2; 

    EXEC InsertIntoCreatureLocations 1, 1;
    EXEC InsertIntoCreatureLocations 2, 2;
COMMIT TRANSACTION

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.031

<span style="color:rgb(60, 64, 67);font-family:Roboto, Arial, sans-serif;letter-spacing:0.2px;">ფუნქციების (Functions) შექმნა/გამოყენება. შენიშვნა: ფუნქციების შექმნისას სავალდებულია გამოიყენოთ T-SQL-ის ინსტრუმენტები (მმართველი კონსტრუქციები, ლოგიკის ოპერატორები, სიმბოლური ტიპის მონაცემებთან მუშაობა, თარიღისა და დროის მონაცემებთან მუშაობა).&nbsp;</span>

In [40]:
CREATE FUNCTION CalculateCreatureAgeDifference (
    @Creature1Age INT,
    @Creature2Age INT
)
RETURNS INT
AS
BEGIN
    DECLARE @AgeDifference INT
    SET @AgeDifference = ABS(@Creature1Age - @Creature2Age)
    RETURN @AgeDifference
END;

Commands completed successfully.

Total execution time: 00:00:00.044

In [41]:
CREATE FUNCTION DetermineSpellPowerStatus (
    @PowerLevel INT,
    @Threshold INT
)
RETURNS NVARCHAR(50)
AS
BEGIN
    DECLARE @Status NVARCHAR(50)
    IF @PowerLevel >= @Threshold
        SET @Status = 'High Power'
    ELSE
        SET @Status = 'Low Power'
    RETURN @Status
END;

Commands completed successfully.

Total execution time: 00:00:00.030

In [42]:
CREATE FUNCTION FormatCreatureDescription (
    @Description NVARCHAR(MAX)
)
RETURNS NVARCHAR(MAX)
AS
BEGIN
    DECLARE @FormattedDescription NVARCHAR(MAX)
    SET @FormattedDescription = '*** ' + @Description + ' ***'
    RETURN @FormattedDescription
END;

Commands completed successfully.

Total execution time: 00:00:00.017

In [43]:
CREATE FUNCTION CalculateAgeFromBirthdate (
    @Birthdate DATETIME
)
RETURNS INT
AS
BEGIN
    DECLARE @Age INT
    SET @Age = DATEDIFF(YEAR, @Birthdate, GETDATE())
    RETURN @Age
END;

Commands completed successfully.

Total execution time: 00:00:00.022

In [44]:
CREATE FUNCTION CheckTerrainClimateMatch (
    @TerrainType NVARCHAR(50),
    @Climate NVARCHAR(50)
)
RETURNS BIT
AS
BEGIN
    DECLARE @Match BIT
    IF @TerrainType = @Climate
        SET @Match = 1
    ELSE
        SET @Match = 0
    RETURN @Match
END;

Commands completed successfully.

Total execution time: 00:00:00.023

In [45]:
CREATE FUNCTION CategorizeSpell (
    @PowerLevel INT
)
RETURNS NVARCHAR(50)
AS
BEGIN
    DECLARE @SpellCategory NVARCHAR(50)

    SET @SpellCategory = 
        CASE 
            WHEN @PowerLevel >= 10 THEN 'High-Level Spell'
            WHEN @PowerLevel >= 5 AND @PowerLevel < 10 THEN 'Medium-Level Spell'
            ELSE 'Low-Level Spell'
        END

    RETURN @SpellCategory
END;

Commands completed successfully.

Total execution time: 00:00:00.027

<span style="color:rgb(60, 64, 67);font-family:Roboto, Arial, sans-serif;letter-spacing:0.2px;">წარმოდგენების (View) შექმნა JOIN კონსტრუქციებისა და ცხრილიდან მონაცემების ამორჩევის ბრძანების ელემენტების გამოყენებით (FROM, WHERE, GROUP BY, HAVING, SELECT, ORDER BY, TOP და სხვ.).&nbsp;</span>

In [48]:
CREATE VIEW CreatureSpellLocationView AS
SELECT TOP 10 
    Creatures.Name AS CreatureName,
    Spells.Name AS SpellName,
    Locations.Name AS LocationName,
    COUNT(*) AS SpellCount,
    AVG(Creatures.Age) AS AverageAge
FROM 
    Creatures
JOIN CreaturesSpells ON Creatures.CreatureID = CreaturesSpells.CreatureID
JOIN Spells ON CreaturesSpells.SpellID = Spells.SpellID
JOIN CreatureLocations ON Creatures.CreatureID = CreatureLocations.CreatureID
JOIN Locations ON CreatureLocations.LocationID = Locations.LocationID
WHERE Creatures.Age > 100
GROUP BY Creatures.Name, Spells.Name, Locations.Name
HAVING COUNT(*) > 1
ORDER BY AverageAge DESC;


: Msg 2714, Level 16, State 3, Procedure CreatureSpellLocationView, Line 1
There is already an object named 'CreatureSpellLocationView' in the database.

Total execution time: 00:00:00.026

In [ ]:
UPDATE c1
SET c1.Age = CASE
    WHEN c2.Age < 50 THEN c1.Age + 2
    ELSE c1.Age - 1
END
FROM Creatures c1
INNER JOIN Creatures c2 ON c1.CreatureID = c2.CreatureID - 1;

- <span style="color: var(--tw-prose-bold); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap; border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600; margin-top: 1.25em;">Incorrect Sequential Logic:</span>  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> This script attempts to join each row in the </span>  `Creatures`  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> table with its "previous" row by matching </span>  `CreatureID - 1`<span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">. However, this assumes that </span>  `CreatureID`  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> are in perfect sequential order with no gaps, which is often not the case in real-world scenarios (due to deletions or other factors).</span>
- <span style="color: var(--tw-prose-bold); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap; border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600; margin-top: 1.25em;">Incomplete Updates:</span>  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> Only creatures whose </span>  `CreatureID`  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> is exactly one more than another creature's </span>  `CreatureID`  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> get updated. This leaves out the first creature (as there's no </span>  `CreatureID - 1`  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> for it) and any creatures following gaps in the </span>  `CreatureID`  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> sequence.</span>
- <span style="color: var(--tw-prose-bold); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap; border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600; margin-top: 1.25em; margin-bottom: 1.25em;">Inability to Track Previous State:</span>  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> The major flaw is that this approach cannot track the changes made to the previous creature within the same operation. It only compares static data from before any updates are made.</span>

In [ ]:
DECLARE @CurrentCreatureID INT, @CurrentAge INT, @PreviousAge INT;

SET @PreviousAge = -1; 

DECLARE creature_cursor CURSOR FOR 
SELECT CreatureID, Age
FROM Creatures
ORDER BY CreatureID;

OPEN creature_cursor;

FETCH NEXT FROM creature_cursor INTO @CurrentCreatureID, @CurrentAge;

WHILE @@FETCH_STATUS = 0
BEGIN
    IF @PreviousAge != -1
    BEGIN
        UPDATE Creatures
        SET Age = CASE
            WHEN @PreviousAge < 50 THEN @CurrentAge + 2
            ELSE @CurrentAge - 1
        END
        WHERE CreatureID = @CurrentCreatureID;
    END

    SET @PreviousAge = @CurrentAge;

    FETCH NEXT FROM creature_cursor INTO @CurrentCreatureID, @CurrentAge;
END

CLOSE creature_cursor;
DEALLOCATE creature_cursor;

<span style="color: var(--tw-prose-bold); border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600; margin-top: 1.25em; margin-bottom: 1.25em; font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">Cursors in SQL Server</span>  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> are database objects used to manipulate data in a set row by row. They provide a means to iterate through a set of rows returned by a query and perform certain operations on each row individually. This is different from the typical set-based operations of SQL where you deal with a batch of rows at a time.</span>

- <span style="border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600; color: var(--tw-prose-bold);">When to Use:</span>
    
    - <span style="color: var(--tw-prose-bold); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap; border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600;">Complex Row-by-Row Processing:</span>  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> Cursors are suitable for operations that require row-by-row manipulation which cannot be efficiently handled with set-based operations.</span>
    - <span style="color: var(--tw-prose-bold); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap; border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600;">Sequential Data Access:</span>  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> Use cursors when data needs to be processed in a specific order, especially when subsequent data processing depends on previous rows.</span>
    - <span style="color: var(--tw-prose-bold); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap; border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600;">Procedural Logic Requirements:</span>  <span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;"> Cursors fit well in scenarios requiring conditional logic and complex decision-making that is not easily implemented in set-based SQL.</span>
- <span style="border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600; color: var(--tw-prose-bold);">When to Avoid:</span>
    
    - <span style="color: var(--tw-prose-bold); border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600;">Large Data Sets:</span> Avoid cursors for large data sets due to performance degradation.
    - <span style="color: var(--tw-prose-bold); border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600;">Simple Queries:</span> For simple data retrieval, updates, or deletions, set-based operations are more efficient.
    - <span style="color: var(--tw-prose-bold); border: 0px solid rgb(217, 217, 227); box-sizing: border-box; --tw-border-spacing-x: 0; --tw-border-spacing-y: 0; --tw-translate-x: 0; --tw-translate-y: 0; --tw-rotate: 0; --tw-skew-x: 0; --tw-skew-y: 0; --tw-scale-x: 1; --tw-scale-y: 1; --tw-pan-x: ; --tw-pan-y: ; --tw-pinch-zoom: ; --tw-scroll-snap-strictness: proximity; --tw-gradient-from-position: ; --tw-gradient-via-position: ; --tw-gradient-to-position: ; --tw-ordinal: ; --tw-slashed-zero: ; --tw-numeric-figure: ; --tw-numeric-spacing: ; --tw-numeric-fraction: ; --tw-ring-inset: ; --tw-ring-offset-width: 0px; --tw-ring-offset-color: #fff; --tw-ring-color: rgba(69,89,164,0.5); --tw-ring-offset-shadow: 0 0 transparent; --tw-ring-shadow: 0 0 transparent; --tw-shadow: 0 0 transparent; --tw-shadow-colored: 0 0 transparent; --tw-blur: ; --tw-brightness: ; --tw-contrast: ; --tw-grayscale: ; --tw-hue-rotate: ; --tw-invert: ; --tw-saturate: ; --tw-sepia: ; --tw-drop-shadow: ; --tw-backdrop-blur: ; --tw-backdrop-brightness: ; --tw-backdrop-contrast: ; --tw-backdrop-grayscale: ; --tw-backdrop-hue-rotate: ; --tw-backdrop-invert: ; --tw-backdrop-opacity: ; --tw-backdrop-saturate: ; --tw-backdrop-sepia: ; font-weight: 600;">Performance-Critical Applications:</span> In applications where performance is a critical factor, cursors should be avoided as they are slower compared to set-based operations.

#### Types of Cursors in SQL Server:

- <span style="color: var(--tw-prose-bold); font-weight: 600; font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">Static Cursor</span>
- <span style="color: var(--tw-prose-bold); font-weight: 600; font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">Dynamic Cursor</span>
- <span style="color: var(--tw-prose-bold); font-weight: 600; font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">Keyset Cursor</span>
- <span style="color: var(--tw-prose-bold); font-weight: 600; font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">Fast Forward-Only Cursor</span>

<span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">A static cursor provides a static copy of the result set. Changes made to the database after the cursor is opened are not reflected in the data returned by fetches made to the cursor.</span>

In [ ]:
DECLARE @CreatureID INT, @Name NVARCHAR(100);

DECLARE static_cursor CURSOR STATIC FOR
SELECT CreatureID, Name
FROM Creatures;

OPEN static_cursor;

FETCH NEXT FROM static_cursor INTO @CreatureID, @Name;

WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT 'Creature ID: ' + CAST(@CreatureID AS NVARCHAR(10)) + ', Name: ' + @Name;
    FETCH NEXT FROM static_cursor INTO @CreatureID, @Name;
END

CLOSE static_cursor;
DEALLOCATE static_cursor;

<span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">A dynamic cursor reflects all changes made to the rows in its result set as you scroll around the cursor.</span>

In [ ]:
DECLARE @SpellID INT, @SpellName NVARCHAR(100);

DECLARE dynamic_cursor CURSOR DYNAMIC FOR
SELECT SpellID, Name
FROM Spells;

OPEN dynamic_cursor;

FETCH NEXT FROM dynamic_cursor INTO @SpellID, @SpellName;

WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT 'Spell ID: ' + CAST(@SpellID AS NVARCHAR(10)) + ', Name: ' + @SpellName;
    FETCH NEXT FROM dynamic_cursor INTO @SpellID, @SpellName;
END

CLOSE dynamic_cursor;
DEALLOCATE dynamic_cursor;

In [ ]:
DECLARE @CreatureID INT, @Name NVARCHAR(100);

DECLARE creature_cursor CURSOR DYNAMIC FOR
SELECT CreatureID, Name
FROM Creatures;

OPEN creature_cursor;

-- Move forward in the cursor
FETCH NEXT FROM creature_cursor INTO @CreatureID, @Name;
WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT 'Forward - Creature ID: ' + CAST(@CreatureID AS NVARCHAR(10)) + ', Name: ' + @Name;
    FETCH NEXT FROM creature_cursor INTO @CreatureID, @Name;
END

-- Now, move backward in the cursor
FETCH PRIOR FROM creature_cursor INTO @CreatureID, @Name;
WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT 'Backward - Creature ID: ' + CAST(@CreatureID AS NVARCHAR(10)) + ', Name: ' + @Name;
    FETCH PRIOR FROM creature_cursor INTO @CreatureID, @Name;
END

CLOSE creature_cursor;
DEALLOCATE creature_cursor;

<span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">A keyset cursor is like a dynamic cursor, except that it does not reflect changes that result in the movement of rows in and out of the cursor's keyset.</span>

In [ ]:
DECLARE @LocationID INT, @LocationName NVARCHAR(100);

DECLARE keyset_cursor CURSOR KEYSET FOR
SELECT LocationID, Name
FROM Locations;

OPEN keyset_cursor;

FETCH NEXT FROM keyset_cursor INTO @LocationID, @LocationName;

WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT 'Location ID: ' + CAST(@LocationID AS NVARCHAR(10)) + ', Name: ' + @LocationName;
    FETCH NEXT FROM keyset_cursor INTO @LocationID, @LocationName;
END

CLOSE keyset_cursor;
DEALLOCATE keyset_cursor;

<span style="color: rgb(55, 65, 81); font-family: Söhne, ui-sans-serif, system-ui, -apple-system, &quot;Segoe UI&quot;, Roboto, Ubuntu, Cantarell, &quot;Noto Sans&quot;, sans-serif, &quot;Helvetica Neue&quot;, Arial, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;, &quot;Noto Color Emoji&quot;; font-size: 16px; white-space: pre-wrap;">A fast forward-only cursor is the simplest and fastest cursor. It only moves forward in the result set.</span>

In [ ]:

DECLARE @CreatureType NVARCHAR(50);

DECLARE fast_forward_cursor CURSOR FAST_FORWARD FOR
SELECT DISTINCT Type
FROM Creatures;

OPEN fast_forward_cursor;

FETCH NEXT FROM fast_forward_cursor INTO @CreatureType;

WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT 'Creature Type: ' + @CreatureType;
    FETCH NEXT FROM fast_forward_cursor INTO @CreatureType;
END

CLOSE fast_forward_cursor;
DEALLOCATE fast_forward_cursor;

## Triggers

Before Insert Trigger

In [ ]:
CREATE TRIGGER BeforeCreatureInsert
ON Creatures
BEFORE INSERT
AS
BEGIN
    -- Prevent inserting creatures with age less than 0
    IF EXISTS (SELECT 1 FROM inserted WHERE Age < 0)
    BEGIN
        RAISERROR('Age cannot be less than 0.', 16, 1)
        ROLLBACK TRANSACTION;
    END
END;


After Update Trigger

In [ ]:
CREATE TRIGGER AfterCreatureUpdate
ON Creatures
AFTER UPDATE
AS
BEGIN
    -- Log the updates to a separate table
    INSERT INTO CreatureUpdates (CreatureID, UpdatedAt)
    SELECT CreatureID, GETDATE() FROM inserted;
END;

Instead of Delete Trigger

In [ ]:
CREATE TRIGGER InsteadOfCreatureDelete
ON Creatures
INSTEAD OF DELETE
AS
BEGIN
    -- Soft delete by updating a flag instead of physically deleting the record
    UPDATE Creatures
    SET IsDeleted = 1
    WHERE CreatureID IN (SELECT CreatureID FROM deleted);
END;

After Insert Trigger

In [ ]:
CREATE TRIGGER AfterCreatureLocationInsert
ON CreatureLocations
AFTER INSERT
AS
BEGIN
    -- Update a related table after a new record is inserted
    UPDATE Locations
    SET CreatureCount = CreatureCount + 1
    FROM Locations l
    INNER JOIN inserted i ON l.LocationID = i.LocationID;
END;

After insert trigger with cursor

In [ ]:
CREATE TRIGGER AfterSpellInsert
ON Spells
AFTER INSERT
AS
BEGIN
    DECLARE @SpellID INT;

    DECLARE SpellCursor CURSOR FOR
    SELECT SpellID FROM inserted;

    OPEN SpellCursor;
    FETCH NEXT FROM SpellCursor INTO @SpellID;

    WHILE @@FETCH_STATUS = 0
    BEGIN
        INSERT INTO SpellLog (SpellID, LogDate)
        VALUES (@SpellID, GETDATE());

        FETCH NEXT FROM SpellCursor INTO @SpellID;
    END;

    CLOSE SpellCursor;
    DEALLOCATE SpellCursor;
END;